In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os 
import sys
import time
import tensorflow as tf
from tensorflow import keras

E:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
E:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
E:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
E:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is depr

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all,y_train_all),(x_test,y_test) = fashion_mnist.load_data()
x_valid,x_train = x_train_all[:5000],x_train_all[5000:]
y_valid,y_train = y_train_all[:5000],y_train_all[5000:]

In [3]:
#归一化操作  x = (s-u)/std   减去均值，除以方差

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

#x_train:[none,28,28]-->[none,784]
x_train_scaler = scaler.fit_transform(
                            x_train.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)
x_valid_scaler = scaler.fit_transform(
                            x_valid.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)
x_test_scaler = scaler.fit_transform(
                            x_test.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)

In [4]:
#tf.keras.models.Sequential

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]), 
    
    keras.layers.Dense(100,activation="relu"),
    keras.layers.Dense(100,activation="relu"),
    keras.layers.Dense(100,activation="relu"),
    keras.layers.Dense(100,activation="relu"),
    keras.layers.Dense(100,activation="relu"),
    keras.layers.Dense(100,activation="relu"),  
    keras.layers.Dense(100,activation="relu"),
    keras.layers.Dense(100,activation="relu"),
    keras.layers.Dense(100,activation="relu"),
    keras.layers.Dense(100,activation="relu"),
    keras.layers.AlphaDropout(rate=0.5),
    #批归一化
    keras.layers.BatchNormalization(),
    
    keras.layers.Dense(10,activation="softmax")])

model.compile(loss="sparse_categorical_crossentropy",
             optimizer="sgd",
             metrics=["accuracy"])


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
#TensorBoard, EarlyStopping, ModelCheckpoint
logdir = '.\dnn-callbacks'
if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir,"fashion_mnist_model.h5")

history = model.fit(x_train_scaler,y_train,epochs=10,
                   validation_data=(x_valid_scaler,y_valid),
                   
                   callbacks = [
                                keras.callbacks.TensorBoard(logdir),
                                keras.callbacks.ModelCheckpoint(output_model_file,save_best_noly=True),
                                keras.callbacks.EarlyStopping(patience=5,min_delta=1e-3)]
                   )

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
54912/55000 [============================>.] - ETA: 0s - loss: 2.0043 - acc: 0.2816

AttributeError: module 'h5py' has no attribute 'File'

In [ ]:
def plt_learning_curve(history):
    pd.DataFrame(history.history).plot(figsize=(8,5))
    plt.grid(True)
    plt.gca().set_ylim(0,1)
    plt.show()

plt_learning_curve(history)

In [ ]:
model.evaluate(x_test_scaler,y_test)